In [1]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
# !pip install impyute
# !pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
# import impyute as impy
from fancyimpute import SoftImpute#, MatrixFactorization
import pandas as pd
import time
import math
# from missingpy import MissForest
from sklearn.model_selection import train_test_split
from numpy.linalg import norm, inv

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from fancyimpute import SimpleFill, SoftImpute
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [2]:
def pca1(Xm, explain_per = 95):
            eig_vals, eig_vectors = np.linalg.eig(np.cov(Xm,rowvar = False, ddof = 0))
            e_indices = np.argsort(eig_vals)[::-1]
            eigenvectors_sorted = eig_vectors[:,e_indices]
            eigenvectors_sorted
            variance_explained = np.array([100*i/sum(eig_vals) for i in eig_vals])#percentage of variance_explained
            n_take = np.where(np.cumsum(variance_explained)>explain_per) # chose st total percentage of variance_explained > 99%
            n_take = np.min(n_take)+1
            print('n components chosen:', n_take)
            V = eigenvectors_sorted[:,:n_take] 
            Xpr = Xm @ V 
            return np.real(Xpr), V
        
def pca2(B, explain_per = .95):
    # return the projection matrix 
    U, S, Wt = np.linalg.svd(B, full_matrices=False)
    # chose st total percentage of variance_explained >explain_per
    n_take = np.where(np.cumsum(S)/sum(S)>explain_per) 
    n_take = np.min(n_take)+1
    print('n components chosen:', n_take)
    W = Wt.T
    Wr = W[:,:n_take]
    V = np.matmul(Wr, Wr.T)
    return np.matmul(B, V), V 

In [3]:
df_X = pd.read_csv("gen_expression_cancer/data.csv")
X = df_X[df_X.columns[1:]].to_numpy()
X.shape

(801, 20531)

In [4]:
y =  pd.read_csv("gen_expression_cancer/labels.csv")
y = y[['Class']].to_numpy()

In [5]:
def one_iter(clf, clf1, clf2, pca = pca2, explain_per = .95):
    X1, X2, y1, y2 = train_test_split(X,y.reshape((-1,1)),test_size=0.3)
    Xtrain1, Xtest1, ytrain1, ytest1 = train_test_split(X1,y1.reshape((-1,1)),test_size=0.5)
    Xtrain2, Xtest2, ytrain2, ytest2 = train_test_split(X2,y2.reshape((-1,1)),test_size=0.5)            
    Xtrain1[:, :ndel1] = np.nan
    Xtest1[:,:ndel1] = np.nan
    Xtrain2[:, ndel2:] = np.nan
    Xtest2[:,ndel2:] = np.nan
    
    scaler = MinMaxScaler()
    scaler.fit(Xtrain1)
    Xtrain1 = scaler.transform(Xtrain1)
    Xtest1 = scaler.transform(Xtest1)
    scaler.fit(Xtrain2)
    Xtrain2 = scaler.transform(Xtrain2)
    Xtest2 = scaler.transform(Xtest2)    
#     print('Xtrain2.shape:', Xtrain2.shape, 'Xtest2.shape:', Xtest2.shape)
    
    p = X.shape[1]
#     ncommon = p - ndel1 - ndel2
    Xtrain1rb, V1 = pca(Xtrain1[:,ndel2:], explain_per)
    Xtrain2rb, V2 = pca(Xtrain2[:, :ndel1], explain_per)
    Xtest1rb = Xtest1[:,ndel2:] @ V1
    Xtest2rb = Xtest2[:, :ndel1] @ V2    
    Xtrain = np.vstack((np.hstack((Xtrain1[:, :ndel2], Xtrain1rb))
                        , np.hstack((Xtrain2rb, Xtrain2[:,ndel1:]))))
    Xtest = np.vstack((np.hstack((Xtest1[:, :ndel2], Xtest1rb))
                        , np.hstack((Xtest2rb, Xtest2[:,ndel1:]))))    
    ytrain = np.vstack((ytrain1, ytrain2)) 
    ytest = np.vstack((ytest1, ytest2))    
    Xtrain = SoftImpute(verbose = False).fit_transform(Xtrain)
    Xtest = SoftImpute(verbose = False).fit_transform(Xtest)
    Xtrain1 = SoftImpute(verbose = False).fit_transform(Xtrain1)
    Xtrain2 = SoftImpute(verbose = False).fit_transform(Xtrain2)
    Xtest1 = SoftImpute(verbose = False).fit_transform(Xtest1)
    Xtest2 = SoftImpute(verbose = False).fit_transform(Xtest2)

#     print('Xtrain1.shape:', Xtrain1.shape, 'Xtest1.shape:', Xtest1.shape)
#     print('Xtrain2.shape:', Xtrain2.shape, 'Xtest2.shape:', Xtest2.shape)    
    clf.fit(Xtrain, ytrain)
    
    # train clf1, clf2 on D1, D2
    Xtrain1 = Xtrain1[:,ndel1:]
    Xtrain2 = Xtrain2[:,:ndel2]
    Xtest1 = Xtest1[:,ndel1:]
    Xtest2 = Xtest2[:,:ndel2]    
    clf1.fit(Xtrain1, ytrain1)
    clf2.fit(Xtrain2, ytrain2)

    return np.array([np.mean(clf.predict(Xtest[:len(ytest1),:])==ytest1.flatten()), 
                     np.mean(clf.predict(Xtest[len(ytest1):,:])==ytest2.flatten()),
                     clf1.score(Xtest1, ytest1),  clf2.score(Xtest2, ytest2)])    

In [14]:
ndel1, ndel2 = 8000, 10000 #5000, 10000

clf = LogisticRegression(random_state=0)
clf1 = LogisticRegression(random_state=0)
clf2 = LogisticRegression(random_state=0)
res = np.array([one_iter(clf, clf1, clf2) for i in range(100)])

C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


n components chosen: 246
n components chosen: 106


C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), f

n components chosen: 246
n components chosen: 106


C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), f

...

C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), f

n components chosen: 246
n components chosen: 106


C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), f

n components chosen: 246
n components chosen: 106


C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), f

In [15]:
np.mean(res, axis = 0)

array([0.99753571, 0.99652893, 0.99739286, 0.99570248])

In [16]:
np.std(res, axis = 0)

array([0.00270558, 0.00525037, 0.00271311, 0.00601435])

In [17]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf1 = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf2 = make_pipeline(StandardScaler(), SVC(gamma='auto'))
res = np.array([one_iter(clf, clf1, clf2) for i in range(100)])

C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


n components chosen: 245
n components chosen: 106


C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\thung\.conda\envs\tf\

n components chosen: 246
n components chosen: 106


C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\thung\.conda\envs\tf\

...

C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\thung\.conda\envs\tf\

n components chosen: 246
n components chosen: 106


C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\thung\.conda\envs\tf\

n components chosen: 246
n components chosen: 106


C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
np.mean(res, axis = 0)

array([0.99528571, 0.9931405 , 0.96967857, 0.95438017])

In [19]:
np.std(res, axis = 0)

array([0.0035312 , 0.00671967, 0.01165507, 0.02243554])

In [26]:
ndel1, ndel2 = 10000, 10000 #5000, 10000

clf = LogisticRegression(random_state=0)
clf1 = LogisticRegression(random_state=0)
clf2 = LogisticRegression(random_state=0)
res = np.array([one_iter(clf, clf1, clf2) for i in range(100)])

C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


n components chosen: 245
n components chosen: 107


C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), f

n components chosen: 246
n components chosen: 106


C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), f

...

C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), f

n components chosen: 246
n components chosen: 106


C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), f

n components chosen: 246
n components chosen: 106


C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), f

In [27]:
np.mean(res, axis = 0)

array([0.99767857, 0.99628099, 0.99767857, 0.99628099])

In [28]:
np.std(res, axis = 0)

array([0.00244193, 0.00565074, 0.00244193, 0.00565074])

In [29]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf1 = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf2 = make_pipeline(StandardScaler(), SVC(gamma='auto'))
res = np.array([one_iter(clf, clf1, clf2) for i in range(100)])

C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


n components chosen: 246
n components chosen: 106


C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\thung\.conda\envs\tf\

n components chosen: 246
n components chosen: 106


C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\thung\.conda\envs\tf\

...

C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\thung\.conda\envs\tf\

n components chosen: 246
n components chosen: 106


C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\thung\.conda\envs\tf\

n components chosen: 246
n components chosen: 106


C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\thung\.conda\envs\tf\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [30]:
np.mean(res, axis = 0)

array([0.99675   , 0.99495868, 0.97139286, 0.95479339])

In [31]:
np.std(res, axis = 0)

array([0.00267714, 0.00660071, 0.01153406, 0.02131062])

In [32]:
np.mean(res, axis = 0).round(3)

array([0.997, 0.995, 0.971, 0.955])

In [33]:
np.std(res, axis = 0).round(3)

array([0.003, 0.007, 0.012, 0.021])